<a href="https://colab.research.google.com/github/BME6938-SPR2023/final_project/blob/main/PelvicSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [1]:
!pip install -q transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 36.6 MB/s eta 0:00:00


## import libraries

In [91]:
#NEED to delect unused libraries
import os as os
from glob import glob
import shutil
from tqdm import tqdm
import numpy as np
import nibabel as nib
import tensorflow as tf
import matplotlib.pyplot as plt
from torch import nn
import evaluate
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
import cv2
import PIL
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize, rotate
# import SimpleITK as sitk
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import torchvision.transforms as transforms
from PIL import Image
from transformers import SegformerFeatureExtractor,SegformerImageProcessor, AutoImageProcessor, TrainingArguments, Trainer

## HF Login

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
hf_username = "ashiyakatuka11"

## Data preparation: 
#### convert nii to png and remove slices without matching labels
- Resized and rotated images 
- Reduced the dataset by only including images that match with segmentation masks with 1 or both labels
- There is an option for converting from .nii to .png

In [ ]:
#data cleaning 1: use stacked data/ labels to get label list
def clean_data(input_folder, output_folder, image_processor): 
  #first remove matched images without labels

  img_folder_path = '/stacked_sliced/train' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/train'

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path

  labels_paths = []

  img_paths = sorted([os.path.join( img_dir, x) for x in os.listdir( img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))[0]
    label_rename = os.path.splitext(os.path.basename(i_mask))[0]

    Reading_Img_NII = nib.load(i_img)
    Reading_Label_NII = nib.load(i_mask)

    img_arr = Reading_Img_NII.get_fdata()
    mask_arr = Reading_Label_NII.get_fdata()
    
     # Define the new size for the resized image
    new_size = (512, 512)
    
    # Resize the image using the resize() function from skimage
    img_resized = resize(img_arr, new_size, anti_aliasing=True)
    mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

    # Rotate the image using the rotate() function from skimage
    angle = 90
    img_resized = rotate(img_resized , angle)
    mask_resized = rotate(mask_resized, angle)

    # Normalized 
    img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
    pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

    label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
    pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8)) 

    pil_image_T = np.stack([pil_image ] * 3, axis=-1).transpose((2,1,0))
    encoded_inputs = image_processor(pil_image_T, pil_label, return_tensors="pt")

    labels = encoded_inputs["labels"].squeeze().unique()
    if len(labels) > 2:
      labels_paths.append(i_mask )
      print(i_img)
      #save stack slices with labels
      pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{img_folder_path}/{img_rename}.png')
      pil_label.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{lbl_folder_path}/{label_rename}.png')
  return(labels_paths)

In [ ]:
def clean_data2(input_folder,output_folder):
  img_folder_path = '/T2_sliced/test' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/test'
  newlbl_folder_path = '/prostate_femur_sliced/test'

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path
  newlabel_dir = output_folder + lbl_folder_path

  img_paths = sorted([os.path.join(img_dir, x) for x in os.listdir(img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])
  newlabels_paths = sorted([os.path.join(newlabel_dir, x) for x in os.listdir(newlabel_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))[0]
    label_rename = os.path.splitext(os.path.basename(i_mask))[0]
    for new_mask in newlabels_paths:
      newlabel_rename = os.path.splitext(os.path.basename(new_mask))[0]
      if (label_rename == newlabel_rename):
        Reading_Img_NII = nib.load(i_img)
        # Reading_Label_NII = nib.load(i_mask)

        img_arr = Reading_Img_NII.get_fdata()
        # mask_arr = Reading_Label_NII.get_fdata()
        
        # Define the new size for the resized image
        new_size = (512, 512)
        
        # Resize the image using the resize() function from skimage
        img_resized = resize(img_arr, new_size, anti_aliasing=True)
        # mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

        # Rotate the image using the rotate() function from skimage
        angle = 90
        img_resized = rotate(img_resized , angle)
        # mask_resized = rotate(mask_resized, angle)

        img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
        pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

        # label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
        # pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8))

        print(i_img)
        #save stack slices with labels
        pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{img_folder_path}/{img_rename}.png')
        # pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{lbl_folder_path}/{label_rename}.png')
  print("Done")


In [ ]:
folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack" #folder with original images
output_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2" #folder where new images will be saved

In [ ]:
image_processor = SegformerImageProcessor(use_python_backend=False)


In [ ]:
clean_data(folder_path,output_path, image_processor) #Run ONLY TWICE for stacked/label train and stacked/label test


In [ ]:
clean_data2(folder_path,output_path) #Run 4 times for T2 and dCT2  train and T2 and dCT2l test

### .nii reduced

In [ ]:
#data cleaning 1: use stacked data/ labels to get label list
import shutil
def reduced_data(input_folder, output_path, image_processor): 
  #first remove matched images without labels
 
  img_folder_path = '/stacked_sliced/train' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/train'

  # new_folder_path = newfolder/img_folder_path
  # !mkdir /content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced/stacked_sliced/test
  # !mkdir /content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced/stacked_sliced/test

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path

  out_img_dir = output_path + img_folder_path
  out_lbl_dir = output_path + lbl_folder_path

  labels_paths = []

  img_paths = sorted([os.path.join( img_dir, x) for x in os.listdir( img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))
    label_rename = os.path.splitext(os.path.basename(i_mask))

    Reading_Img_NII = nib.load(i_img)
    Reading_Label_NII = nib.load(i_mask)

    img_arr = Reading_Img_NII.get_fdata()
    mask_arr = Reading_Label_NII.get_fdata()
    
     # Define the new size for the resized image
    new_size = (512, 512)
    
    # Resize the image using the resize() function from skimage
    img_resized = resize(img_arr, new_size, anti_aliasing=True)
    mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

    # Rotate the image using the rotate() function from skimage
    angle = 90
    img_resized = rotate(img_resized , angle)
    mask_resized = rotate(mask_resized, angle)

    img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
    pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

    label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
    pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8)) 

    pil_image_T = np.stack([pil_image ] * 3, axis=-1).transpose((2,1,0))
    encoded_inputs = image_processor(pil_image_T, pil_label, return_tensors="pt")

    labels = encoded_inputs["labels"].squeeze().unique()
    if len(labels) > 2:
      labels_paths.append(i_mask )
      print(i_img)
      #save stack slices with labels
      shutil.copy(i_img, out_img_dir )
      # shutil.copy(i_mask,out_lbl_dir )
  return(labels_paths)

In [ ]:
def reduced_data2(input_folder,output_path):
  img_folder_path = '/stacked_sliced/train' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/train'
  newlbl_folder_path = '/prostate_femur_sliced/test'

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path
  newlabel_dir = output_path + lbl_folder_path

  out_img_dir = output_path + img_folder_path
  # out_lbl_dir = output_path + lbl_folder_path

  img_paths = sorted([os.path.join(img_dir, x) for x in os.listdir(img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])
  newlabels_paths = sorted([os.path.join(newlabel_dir, x) for x in os.listdir(newlabel_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))
    label_rename = os.path.splitext(os.path.basename(i_mask))
    for new_mask in newlabels_paths:
      newlabel_rename = os.path.splitext(os.path.basename(new_mask))
      if (label_rename == newlabel_rename):
        Reading_Img_NII = nib.load(i_img)
        # Reading_Label_NII = nib.load(i_mask)

        img_arr = Reading_Img_NII.get_fdata()
        # mask_arr = Reading_Label_NII.get_fdata()
        
        # Define the new size for the resized image
        new_size = (512, 512)
        
        # Resize the image using the resize() function from skimage
        img_resized = resize(img_arr, new_size, anti_aliasing=True)
        # mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

        # Rotate the image using the rotate() function from skimage
        angle = 90
        img_resized = rotate(img_resized , angle)
        # mask_resized = rotate(mask_resized, angle)

        img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
        pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

        # label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
        # pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8))

        print(i_img)
        #save stack slices with labels
        shutil.copy(i_img,out_img_dir )
        # pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{lbl_folder_path}/{label_rename}.png')
  print("Done")

In [ ]:
folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack"
output_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced"

In [ ]:
image_processor = SegformerImageProcessor(use_python_backend=False)

In [ ]:
reduced_data(folder_path,output_path, image_processor) #Run ONLY TWICE for stacked/label train and stacked/label test

In [ ]:
reduced_data2(folder_path,output_path) #Run 4 times for T2 and dCT2  train and T2 and dCT2l test

## Load the data 

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2" #new directory with new .png images 

In [6]:

folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced"  #new directory with new reduced .nii images 

### check labels 

In [8]:
#sample image/label
img_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2/stacked_sliced/train/P3_04_stack_Z48.png"
lbl_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2/prostate_femur_sliced/train/3_04_prostate_femur_Z48.png"
img = Image.open(img_path)
label= Image.open(lbl_path)
label_arr = np.array(label)
print(np.unique(label_arr))

#visualize image/mask

# figure,axis = plt.subplots(1,2,figsize=(10,10))
# # Plot_Color_Op = axis[0].imshow(img)
# axis[0].imshow(img)
# axis[0].set_xlabel(img.size)
# axis[0].set_ylabel(img.size)
# axis[0].set_title("IMAGE")

# axis[1].imshow(label)
# axis[1].set_xlabel(label.size)
# axis[1].set_ylabel(label.size)
# axis[1].set_title("MASK")

[  0 254 255]


In [10]:
#sample image/label
img_path = folder_path + "/stacked_sliced/train/P1_02_stack_Z48.nii"
lbl_path = folder_path + "/prostate_femur_sliced/train/3_04_prostate_femur_Z48.nii"


Reading_Img_NII = nib.load(img_path)
Reading_Label_NII = nib.load(lbl_path)

img_arr = Reading_Img_NII.get_fdata()
mask_arr = Reading_Label_NII.get_fdata()
print(np.unique(mask_arr))

#visualize image/mask

# # Plot_Color_Op = axis[0].imshow(img)
# axis[0].imshow(img)
# axis[0].set_xlabel(img.size)
# axis[0].set_ylabel(img.size)
# axis[0].set_title("IMAGE")

# axis[1].imshow(label)
# axis[1].set_xlabel(label.size)
# axis[1].set_ylabel(label.size)
# axis[1].set_title("MASK")

[0. 2.]


### Get the unique labels

In [11]:
#data cleaning 1: use stacked data/ labels to get label list
def unique_data(input_folder): 

  lbl_folder_path = '/prostate_femur_sliced/test'
  label_dir = input_folder + lbl_folder_path

  labels_paths = []
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])

  for i_mask in labels_paths:
    Reading_Label_NII = nib.load(i_mask)
    mask_arr = Reading_Label_NII.get_fdata()
    
    print(np.unique(mask_arr))
  return("done")

In [12]:
unique_data(folder_path)

[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 2.]
[0. 2.]
[0. 2.]
[0. 2.]
[0. 2.]
[0. 2.]


'done'

### load image/label directories

In [13]:
## directory paths for all datasets

##train
tr_stk_img_dir=folder_path + '/stacked_sliced/train'
tr_t2_img_dir=folder_path + '/T2_sliced/train'
tr_dct_img_dir=folder_path + '/dCT2_sliced/train'
tr_labels_dir=folder_path +'/prostate_femur_sliced/train'
# tr_newlabels_dir=folder_path +'/prostate/train'

##test
ts_stk_img_dir=folder_path + '/stacked_sliced/test'
ts_t2_img_dir=folder_path + '/T2_sliced/test'
ts_dct_img_dir=folder_path + '/dCT2_sliced/test'
ts_labels_dir=folder_path +'/prostate_femur_sliced/test'

In [14]:
tr_stk_img_paths = sorted([os.path.join(tr_stk_img_dir, x) for x in os.listdir(tr_stk_img_dir)])
tr_t2_img_paths = sorted([os.path.join(tr_t2_img_dir, x) for x in os.listdir(tr_t2_img_dir)])
tr_dct_img_paths = sorted([os.path.join(tr_dct_img_dir, x) for x in os.listdir(tr_dct_img_dir)])
tr_labels_paths = sorted([os.path.join(tr_labels_dir, x) for x in os.listdir(tr_labels_dir)])
# tr_newlabels_paths = sorted([os.path.join(tr_newlabels_dir, x) for x in os.listdir(tr_newlabels_dir)])

ts_stk_img_paths = sorted([os.path.join(ts_stk_img_dir, x) for x in os.listdir(ts_stk_img_dir)])
ts_t2_img_paths = sorted([os.path.join(ts_t2_img_dir, x) for x in os.listdir(ts_t2_img_dir)])
ts_dct_img_paths = sorted([os.path.join(ts_dct_img_dir, x) for x in os.listdir(ts_dct_img_dir)])
ts_labels_paths = sorted([os.path.join(ts_labels_dir, x) for x in os.listdir(ts_labels_dir)])

In [15]:
#check to make sure all images match with labels
print(len(tr_stk_img_paths))
print(len(tr_t2_img_paths))
print(len(tr_dct_img_paths))
print(len(tr_labels_paths))
print("***")
print(len(ts_stk_img_paths))
print(len(ts_t2_img_paths))
print(len(ts_dct_img_paths))
print(len(ts_labels_paths))

299
299
299
299
***
51
51
51
51


## Data Preprocessing
To make them compatible with the SegFormer model from Hugging Face Transformers, we:

- Normalize the images with the mean and standard deviation used during pre-training SegFormer.
- Transpose the images such that they are in "channels_first" format. 

### pelvic segmentation class

In [127]:
### .nii
class NiftiPelvicSegmentationDataset(torch.utils.data.Dataset):
    """Image (pelvic) segmentation dataset."""

    def __init__(self, img_paths, mask_paths, train=True):
        """
        Args:

        """
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        # self.transform = transform
        self.train = train

        assert len(self.img_paths) == len(self.mask_paths), "check that number of images equals number of segmentation masks "

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx): 

        #Load the Nifti image      
        img = nib.load(self.img_paths[idx])
        mask = nib.load(self.mask_paths[idx])

        # Convert the Nifti image to a NumPy array
        img_arr = img.get_fdata()
        mask_arr = mask.get_fdata()

        # Define the new size for the resized image
        if (img_arr.shape != (512, 512)):
          new_size = (512, 512)
          # Resize the image using the resize() function from skimage
          img_arr= resize(img_arr, new_size, anti_aliasing=True)
          mask_arr = resize(mask_arr, new_size, anti_aliasing=True)

        #Normalize the image
        img_arr = (img_arr - img_arr.min()) / (img_arr.max() - img_arr.min())
        img_arr = np.stack([img_arr ] * 3, axis=-1).transpose((2,1,0))

        # if self.transform is not None:
        #     img_arr = self.transform(img_arr)
        #     mask_arr = self.transform(mask_arr)

        # Convert to PyTorch tensors
        img_tensor = torch.from_numpy(img_arr).unsqueeze(0) # Add channel dimension
        mask_tensor = torch.from_numpy(mask_arr).unsqueeze(0) # Add channel dimension

        # pad both image and segmentation map to same size
        inputs = {"pixel_values": img_tensor.squeeze_().float(), "labels": mask_tensor.squeeze_().long()}

        return inputs

In [128]:
train_dataset = NiftiPelvicSegmentationDataset(img_paths=tr_stk_img_paths, mask_paths=tr_labels_paths)
valid_dataset = NiftiPelvicSegmentationDataset(img_paths=ts_stk_img_paths, mask_paths=ts_labels_paths, train=False)

### check data 

In [129]:
print("# of training data:", len(train_dataset))
print("# of test data:", len(valid_dataset))

# of training data: 299
# of test data: 51


In [130]:
encoded_inputs = train_dataset[0]
print(encoded_inputs["pixel_values"].shape)
print(encoded_inputs["labels"].shape)
print(encoded_inputs["labels"].squeeze().unique())

torch.Size([3, 512, 512])
torch.Size([512, 512])
tensor([0, 1])


### Get the labels 

In [ ]:
# unique_labels = set()
unique_pixels_list = []
#train data
for i in range(len(train_dataset)):
  unique_pixels = train_dataset[i]["labels"].squeeze().unique()
  if str(unique_pixels) not in unique_pixels_list:
    unique_pixels_list.append(str(unique_pixels))

#test data
for i in range(len(valid_dataset)):
  unique_pixels = valid_dataset[i]["labels"].squeeze().unique()
  if str(unique_pixels) not in unique_pixels_list:
    unique_pixels_list.append(str(unique_pixels))

print(len(unique_pixels_list))


3


In [ ]:
batch["labels"].shape

## Finetuning the model

In [173]:
#set labels
import json
from huggingface_hub import hf_hub_download

# hf_dataset_identifier = "Cats-Can/image_data"

# filename = "id2label.json"
# id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))
id2label = {0: 'Background',  1: 'Prostate', 2: 'Femur'} #uncomment to use if having problems with HF
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)
print("Id2label:", id2label)
print(num_labels)

Id2label: {0: 'Background', 1: 'Prostate', 2: 'Femur'}
3


In [174]:
# load model
from transformers import SegformerForSemanticSegmentation

In [175]:
# define model
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0",
                                                         num_labels=3, 
                                                         id2label=id2label, 
                                                         label2id=label2id,
)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.2.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.linear_c.

In [145]:
from datasets import load_metric

metric = load_metric("mean_iou", "erntkn/dice_coefficient")

In [ ]:
from datasets import list_metrics
list_metrics()

In [176]:
from transformers import TrainingArguments

epochs = 1
lr = 0.00006
batch_size = 4

hub_model_id = "segformer-b0-finetuned-pelvic-segmentation"

training_args = TrainingArguments(
    "segformer-b0-finetuned-pelvic-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    # push_to_hub=True,
    # hub_model_id=hub_model_id,
    # hub_strategy="end",
)

In [177]:
from sklearn.metrics import precision_score,recall_score,f1_score

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)

    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,

        )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()   
    per_category_iou = metrics.pop("per_category_iou").tolist()



    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

    return metrics

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()